In [8]:
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import datasets
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

import pandas as pd
iris = datasets.load_iris()
X = iris.data[:, :2]
y = iris.target
df = pd.read_csv('/content/Social_Network_Ads.csv')

# Add feature columns for input and output
feature_cols = ['Gender','Age','EstimatedSalary'] # Social Network Dataset

# Encode the categorical features
df['Gender'].replace(to_replace=['Male', 'Female'], value=[1,2], inplace=True) # Social Network Dataset

X = df[feature_cols]
y = df['Purchased']



X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)



**Decission Tree Classification**

In [74]:
import pandas as pd
from sklearn import model_selection
'''
Authors: Ashwani Kashyap, Anshul Pardhi
'''

import math

def unique_vals(rows, col):
    """Find the unique values for a column in a dataset."""
    return set([row[col] for row in rows])

def class_counts(rows):
    """Counts the number of each type of example in a dataset."""
    counts = {}  # a dictionary of label -> count.
    for row in rows:
        # in our dataset format, the label is always the last column
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts


def max_label(dict):
    max_count = 0
    label = ""

    for key, value in dict.items():
        if dict[key] > max_count:
            max_count = dict[key]
            label = key

    return label




def is_numeric(value):
    """Test if a value is numeric."""
    return isinstance(value, int) or isinstance(value, float)



class Question:
    """A Question is used to partition a dataset.

    This class just records a 'column number' (e.g., 0 for Color) and a
    'column value' (e.g., Green). The 'match' method is used to compare
    the feature value in an example to the feature value stored in the
    question. See the demo below.
    """

    def __init__(self, column, value, header):
        self.column = column
        self.value = value
        self.header = header

    def match(self, example):
        # Compare the feature value in an example to the
        # feature value in this question.
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value

    def __repr__(self):
        # This is just a helper method to print
        # the question in a readable format.
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (
            self.header[self.column], condition, str(self.value))


def partition(rows, question):
    """Partitions a dataset.

    For each row in the dataset, check if it matches the question. If
    so, add it to 'true rows', otherwise, add it to 'false rows'.
    """
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows


def gini(rows):
    """Calculate the Gini Impurity for a list of rows.

    There are a few different ways to do this, I thought this one was
    the most concise. See:
    https://en.wikipedia.org/wiki/Decision_tree_learning#Gini_impurity
    """
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity

def entropy(rows):

    # compute the entropy.
    entries = class_counts(rows)
    avg_entropy = 0
    size = float(len(rows))
    for label in entries:
        prob = entries[label] / size
        avg_entropy = avg_entropy + (prob * math.log(prob, 2))
    return -1*avg_entropy


def info_gain(left, right, current_uncertainty):
    """Information Gain.

    The uncertainty of the starting node, minus the weighted impurity of
    two child nodes.
    """
    p = float(len(left)) / (len(left) + len(right))

    ## TODO: Step 3, Use Entropy in place of Gini
    return current_uncertainty - p * entropy(left) - (1 - p) * entropy(right)

def split_ratio(left, right):
    d1 = float(len(left)) / (len(left) + len(right))
    d2 = 1 - d1
    if d1== 0 or d2 ==0:
        return -1
    return -1*(d1*math.log(d1,2)+ d2*math.log(d2,2))

def gain_ratio(info_gain, split_ratio):
    return float(info_gain/split_ratio)

def find_best_split(rows, header, metric):
    """Find the best question to ask by iterating over every feature / value
    and calculating the information gain."""
    best_gain = 0  # keep track of the best information gain
    best_question = None  # keep train of the feature / value that produced it
    current_uncertainty = entropy(rows)
    n_features = len(rows[0]) - 1  # number of columns
    rows_len = len(rows)
    gini_total_dataset = gini(rows)

    for col in range(n_features):  # for each feature

        values = set([row[col] for row in rows])  # unique values in the column

        for val in values:  # for each value

            question = Question(col, val, header)

            # try splitting the dataset
            left, right = partition(rows, question)
            if metric == "gini":   
                gini_left = gini(left)
                gini_right = gini(right)
                gini_of_feature = ((len(left)/ rows_len) * gini_left) + ((len(right)/rows_len) * gini_right)
                gini_delta = gini_total_dataset - gini_of_feature
                if(gini_delta >= best_gain):
                    best_gain, best_question = gini_delta, question
            elif metric == "info_gain":
                gain = info_gain(left, right, current_uncertainty)
                if gain >= best_gain:
                    best_gain, best_question = gain, question
            else: # Gain Ratio
                gain_ratio_cal = gain_ratio(info_gain(left, right, current_uncertainty), split_ratio(left, right))
                if gain_ratio_cal >= best_gain:
                    best_gain, best_question = gain_ratio_cal, question

            # Skip this split if it doesn't divide the
            # dataset.
            if len(left) == 0 or len(right) == 0:
                continue

    return best_gain, best_question

## TODO: Step 2
class Leaf:
    """A Leaf node classifies data.

    This holds a dictionary of class (e.g., "Apple") -> number of times
    it appears in the rows from the training data that reach this leaf.
    """

    def __init__(self, rows, id, depth):
        self.predictions = class_counts(rows)
        self.predicted_label = max_label(self.predictions)
        self.id = id
        self.depth = depth

## TODO: Step 1
class Decision_Node:
    """A Decision Node asks a question.

    This holds a reference to the question, and to the two child nodes.
    """

    def __init__(self,
                 question,
                 true_branch,
                 false_branch,
                 depth,
                 id,
                 rows):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch
        self.depth = depth
        self.id = id
        self.rows = rows


## TODO: Step 3
def build_tree(rows, header, depth=0, id=0):
    """Builds the tree.

    Rules of recursion: 1) Believe that it works. 2) Start by checking
    for the base case (no further information gain). 3) Prepare for
    giant stack traces.
    """

    gain, question = find_best_split(rows, header, "gain_ratio")

    if gain == 0:
        return Leaf(rows, id, depth)

    true_rows, false_rows = partition(rows, question)

    # Recursively build the true branch.
    true_branch = build_tree(true_rows, header, depth + 1, 2 * id + 2)

    # Recursively build the false branch.
    false_branch = build_tree(false_rows, header, depth + 1, 2 * id + 1)

    return Decision_Node(question, true_branch, false_branch, depth, id, rows)

## TODO: Step 8 - already done for you
def prune_tree(node, prunedList):
    """Builds the tree.

    Rules of recursion: 1) Believe that it works. 2) Start by checking
    for the base case (no further information gain). 3) Prepare for
    giant stack traces.
    """

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        return node
    # If we reach a pruned node, make that node a leaf node and return. Since it becomes a leaf node, the nodes
    # below it are automatically not considered
    if int(node.id) in prunedList:
        return Leaf(node.rows, node.id, node.depth)

    # Call this function recursively on the true branch
    node.true_branch = prune_tree(node.true_branch, prunedList)

    # Call this function recursively on the false branch
    node.false_branch = prune_tree(node.false_branch, prunedList)

    return node

## TODO: Step 6
def classify(row, node):
    """See the 'rules of recursion' above."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        return node.predicted_label

    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

## TODO: Step 4
def print_tree(node, spacing=""):
    """World's most elegant tree printing function."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        print(spacing + "Leaf id: " + str(node.id) + " Predictions: " + str(node.predictions) + " Label Class: " + str(node.predicted_label))
        return

    # Print the question at this node
    print(spacing + str(node.question) + " id: " + str(node.id) + " depth: " + str(node.depth))

    # Call this function recursively on the true branch
    print(spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # Call this function recursively on the false branch
    print(spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")


def print_leaf(counts):
    """A nicer way to print the predictions at a leaf."""
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs

def getLeafNodes(node, leafNodes =[]):

    # Base case
    if isinstance(node, Leaf):
        leafNodes.append(node)
        return

    # Recursive right call for true values
    getLeafNodes(node.true_branch, leafNodes)

    # Recursive left call for false values
    getLeafNodes(node.false_branch, leafNodes)

    return leafNodes


def getInnerNodes(node, innerNodes =[]):

    # Base case
    if isinstance(node, Leaf):
        return

    innerNodes.append(node)

    # Recursive right call for true values
    getInnerNodes(node.true_branch, innerNodes)

    # Recursive left call for false values
    getInnerNodes(node.false_branch, innerNodes)

    return innerNodes

## TODO: Step 6
def computeAccuracy(rows, node):

    count = len(rows)
    if count == 0:
        return 0

    accuracy = 0
    for row in rows:
        # last entry of the column is the actual label
        if row[-1] == classify(row, node):
            accuracy += 1
    return round(accuracy/count, 2)


# default data set
df = pd.read_csv('/content/Social_Network_Ads copy.csv')
# df = pd.read_csv('data_set/contraceptive-method-choice.csv')
# df = pd.read_csv('data_set/xor-gate-3-inputs.csv')
header = list(df.columns)

lst = df.values.tolist()

trainDF, testDF = model_selection.train_test_split(lst, test_size=0.2)
t = build_tree(trainDF, header)


# get leaf and inner nodes
print("\nLeaf nodes ****************")
leaves = getLeafNodes(t)
for leaf in leaves:
    print("id = " + str(leaf.id) + " depth =" + str(leaf.depth))

print("\nNon-leaf nodes ****************")
innerNodes = getInnerNodes(t)

for inner in innerNodes:
    print("id = " + str(inner.id) + " depth =" + str(inner.depth))

# print tree
maxAccuracy = computeAccuracy(testDF, t)
print("\nTree before pruning with accuracy: " + str(maxAccuracy*100) + "\n")
print_tree(t)



Leaf nodes ****************
id = 254 depth =7
id = 1018 depth =9
id = 2036 depth =10
id = 4072 depth =11
id = 8144 depth =12
id = 16288 depth =13
id = 32576 depth =14
id = 32575 depth =14
id = 1016 depth =9
id = 1015 depth =9
id = 252 depth =7
id = 251 depth =7
id = 124 depth =6
id = 123 depth =6
id = 29 depth =4
id = 13 depth =3
id = 12 depth =3
id = 24 depth =4
id = 23 depth =4
id = 46 depth =5
id = 45 depth =5
id = 21 depth =4
id = 9 depth =3
id = 18 depth =4
id = 302 depth =8
id = 301 depth =8
id = 149 depth =7
id = 73 depth =6
id = 35 depth =5
id = 142 depth =7
id = 141 depth =7
id = 69 depth =6
id = 33 depth =5
id = 270 depth =8
id = 1082 depth =10
id = 1081 depth =10
id = 539 depth =9
id = 133 depth =7
id = 65 depth =6
id = 64 depth =6
id = 258 depth =8
id = 257 depth =8
id = 127 depth =7

Non-leaf nodes ****************
id = 0 depth =0
id = 2 depth =1
id = 6 depth =2
id = 14 depth =3
id = 30 depth =4
id = 62 depth =5
id = 126 depth =6
id = 253 depth =7
id = 508 depth =8
id = 1

**SVM Classification**

In [9]:
clf = svm.SVC()
# instantiate classifier with default hyperparameters
clf = svm.SVC(decision_function_shape='ovo')
clf.fit(X_train, y_train)


# make predictions on test set
y_pred= clf.predict(X_test)


# compute and print accuracy score
print('Model accuracy score with default hyperparameters: {0:0.4f}'. format(accuracy_score(y_test, y_pred)))
print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

Model accuracy score with default hyperparameters: 0.8250
0.825
[[56  2]
 [12 10]]
              precision    recall  f1-score   support

           0       0.82      0.97      0.89        58
           1       0.83      0.45      0.59        22

    accuracy                           0.82        80
   macro avg       0.83      0.71      0.74        80
weighted avg       0.83      0.82      0.81        80



**Deep Learning Classification**

In [10]:
import tensorflow as tf
from tensorflow.keras import layers
import pandas as pd
import numpy as np
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.utils import to_categorical

from sklearn.preprocessing import LabelEncoder
encoder =  LabelEncoder()
y1 = encoder.fit_transform(y)
Y = pd.get_dummies(y1).values
print(Y[0:5])
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
  ])
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=50, epochs=100)
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', loss)
print('Test accuracy:', accuracy)
y_pred = model.predict(X_test)
y_pred=np.argmax(y_pred, axis=1)
y_test=np.argmax(y_test, axis=1)
cm = confusion_matrix(y_test, y_pred)
print(cm)
#print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]]
Epoch 1/100
7/7 [==============================] - 1s 3ms/step - loss: 14246.2812 - accuracy: 0.3781
Epoch 2/100
7/7 [==============================] - 0s 3ms/step - loss: 11531.0400 - accuracy: 0.3781
Epoch 3/100
7/7 [==============================] - 0s 3ms/step - loss: 9729.1973 - accuracy: 0.3781
Epoch 4/100
7/7 [==============================] - 0s 3ms/step - loss: 8095.8179 - accuracy: 0.3781
Epoch 5/100
7/7 [==============================] - 0s 3ms/step - loss: 6540.4097 - accuracy: 0.3781
Epoch 6/100
7/7 [==============================] - 0s 3ms/step - loss: 5072.0586 - accuracy: 0.3781
Epoch 7/100
7/7 [==============================] - 0s 3ms/step - loss: 3613.5879 - accuracy: 0.3781
Epoch 8/100
7/7 [==============================] - 0s 3ms/step - loss: 2228.9385 - accuracy: 0.3781
Epoch 9/100
7/7 [==============================] - 0s 3ms/step - loss: 813.2925 - accuracy: 0.3781
Epoch 10/100
7/7 [==============================] - 0s 4ms/step

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
